# Phink correlassion and feature engineering for competition TPS jan 2022



In [ ]:
# Install libraries
!pip install holidays
import holidays
import pandas as pd
import numpy  as np
import copy
import phik

In [ ]:
# Load data
# Convert the date column to datetime.
train = pd.read_csv(
    '/kaggle/input/tabular-playground-series-jan-2022/train.csv', parse_dates=['date'])
test = pd.read_csv(
    '/kaggle/input/tabular-playground-series-jan-2022/test.csv',parse_dates=['date'])
sample     = pd.read_csv('../input/tabular-playground-series-jan-2022/sample_submission.csv')

In [ ]:
# Create some simple features base on the Date field...

def create_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Create features base on the date variable, the idea is to extract as much 
    information from the date componets.
    Args
        df: Input data to create the features.
    Returns
        df: A DataFrame with the new time base features.
    """

    holiday_FI = holidays.CountryHoliday('FI', years=[2015, 2016, 2017, 2018, 2019])
    holiday_NO = holidays.CountryHoliday('NO', years=[2015, 2016, 2017, 2018, 2019])
    holiday_SE = holidays.CountryHoliday('SE', years=[2015, 2016, 2017, 2018, 2019])

    holiday_dict = holiday_FI.copy()
    holiday_dict.update(holiday_NO)
    holiday_dict.update(holiday_SE)
    
    # Start the creating future process.
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['dayofweek'] = df['date'].dt.dayofweek
    df['dayofmonth'] = df['date'].dt.days_in_month
    df['dayofyear'] = df['date'].dt.dayofyear
    df['weekofyear'] = df['date'].dt.week
    df['weekday'] = df['date'].dt.weekday
    df['weekend'] = np.where((df['dayofweek'] == 5) | (df['dayofweek'] == 6), 1, 0)
    df['crism'] = np.where((df['day'] == 24) & (df['month'] == 12), 1, 0)
    df['newyear'] = np.where((df['day'] == 31) & (df['month'] == 12), 1, 0)
    df['newyear'] = np.where((df['day'] == 1) & (df['month'] == 1), 1, 0)
    
    df['holiday_name'] = df['date'].map(holiday_dict)
    df['is_holiday'] = np.where(df['holiday_name'].notnull(), 1, 0)
    df['holiday_name'] = df['holiday_name'].fillna('Not Holiday')   
    
    
    return df

In [ ]:
train = create_time_features(train)
test = create_time_features(test)

In [ ]:
#===========================================================================
# select some features add target
#===========================================================================
features = ['country',
            'store',
            'product',
            'year',            
            'month',
            'day',
#             'dayofweek', and 'weekday' have the same meaning
            'dayofmonth',            
            'dayofyear', 
            'weekofyear',
            'weekday', 
            'weekend', 
            'crism',
            'newyear',
            'holiday_name',
            'is_holiday']
target = ['num_sold']

In [ ]:
# create and display the Phink correlation matrix
phik_overview_features = train[target+features].phik_matrix()
phik_overview_features.round(2)

**Looking ahead, I can say that these features are not enough for a high-quality model.**

**I propose to discuss ideas for creating new features on this basis, or creating new ones.**

**Write suggestions in the comments.**